In [ ]:
import os
from os import path
from astropy.time import Time
from astropy.io import fits, ascii
import astropy.units as u
from astropy.table import Table, QTable
from astropy.constants import G

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import h5py
from sqlalchemy import func
from sqlalchemy.orm.exc import NoResultFound
from scipy.optimize import root
from scipy.stats import scoreatpercentile
import tqdm
import schwimmbad
from scipy.optimize import root

from thejoker import JokerSamples
from thejoker.sampler import JokerParams, TheJoker
from thejoker.plot import plot_rv_curves

from twoface.config import TWOFACE_CACHE_PATH
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, 
                        StarResult, Status, JokerRun, initialize_db)
from twoface.data import APOGEERVData
from twoface.plot import plot_data_orbits
from twoface.ext import FileLoader

In [ ]:
Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter.hdf5')
mcmc_samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter-mcmc.hdf5')

In [ ]:
allvisit_dr14 = fits.getdata('/Users/adrian/data/APOGEE_DR14/allVisit-l31c.2.fits')

In [ ]:
allvisit = fits.getdata('/Users/adrian/data/APOGEE_DR15beta/allVisit-t9-l31c-58104.fits')
allstar = fits.getdata('/Users/adrian/data/APOGEE_DR15beta/allStar-t9-l31c-58104.fits')

In [ ]:
sheet = FileLoader.GoogleSheets('1z0ukn8QWJL7dTrPdCFWlhQ6r-X-kwiA4z0YRqalZsY4', 1633658731)
candidates = sheet.load()
candidates[10]

In [ ]:
dr14_t = Time('2016-07-01')

In [ ]:
with h5py.File(samples_file) as f:
    for apogee_id in candidates['APOGEE_ID']:
        print(apogee_id)
        try:
            star = AllStar.get_apogee_id(session, apogee_id)
            dr14_data = star.apogeervdata()
            samples = JokerSamples.from_hdf5(f[apogee_id])
        except NoResultFound:
            dr14_data = APOGEERVData.from_visits(allvisit_dr14[allvisit_dr14['APOGEE_ID'] == apogee_id])
            samples = None
            
        visits = allvisit[allvisit['APOGEE_ID'] == apogee_id]
        visits = visits[visits['VRELERR'] < 100.]
        dr15_data = APOGEERVData.from_visits(visits)
        
        tbl = QTable()
        tbl['time'] = dr15_data.t
        tbl['rv'] = dr15_data.rv
        tbl['rv_err'] = dr15_data.stddev
        tbl.write('../../data/bh-candidates/{0}.ecsv'.format(apogee_id), format='ascii.ecsv')
        
        h = np.ptp(dr15_data.t.mjd)
        tgrid = np.linspace(dr15_data.t.mjd.min()-0.1*h,
                            dr15_data.t.mjd.max()+0.1*h,
                            4096)
        
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
        dr14_data.plot(ax=ax, color='tab:red', markersize=6)
        dr15_data.plot(ax=ax, color='k', markersize=4)
        if samples is not None:
            _ = plot_rv_curves(samples, tgrid, n_plot=128, ax=ax, 
                               plot_kwargs=dict(color='tab:blue', linewidth=1))
        ax.set_title(apogee_id)
        fig.tight_layout()
        fig.savefig('../../plots/for-marla/{0}-dr14-dr15.pdf'.format(apogee_id))